In [ ]:
import scanpy as sc
import torch
from model import VAE
"""
# Load the data
adata_path = "/mnt/data2/home/leonardo/git/multilineage_velocity/pancreas_test_2499/pancreas/K11/adata/adata_K11_dt_ve.h5ad"
pan = sc.read_h5ad(adata_path)
"""

In [ ]:
gastrulation_path = "/mnt/data2/home/leonardo/git/multilineage_velocity/benchmark/imVelo/pancreas/imVelo_pancreas.h5ad"
adata = sc.read_h5ad(gastrulation_path)


In [ ]:
gene_name = "Snrnp70"

In [ ]:
sc.pl.scatter(adata, x="isomap_1", y=f"{gene_name}", layers="velocity_u", color="clusters",
              title="unspliced velocity over time")

sc.pl.scatter(adata, x="isomap_1", y=f"{gene_name}", layers="velocity", color="clusters",
              title="spliced velocity over time")

sc.pl.scatter(adata, x="isomap_1", y=f"{gene_name}", layers="Mu", color="clusters",
              title="Unspliced expression over time")

sc.pl.scatter(adata, x="isomap_1", y=f"{gene_name}", layers="Ms", color="clusters",
              title="Spliced expression over time")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Retrieve the data
x = adata.obs["isomap_1"]
y_velocity_u = adata.layers["velocity_u"][:, adata.var_names == gene_name].flatten()
y_Mu = adata.layers["Mu"][:, adata.var_names == gene_name].flatten()

# Get the cluster labels and corresponding colors
clusters = adata.obs["clusters"]
cluster_colors = adata.uns["clusters_colors"]

# Map clusters to colors
cluster_color_map = {cluster: color for cluster, color in zip(clusters.cat.categories, cluster_colors)}
colors = clusters.map(cluster_color_map)

# Plot both curves
plt.figure(figsize=(8, 6))

# Scatter for unspliced velocity with cluster-based colors
plt.scatter(x, y_velocity_u, label="Unspliced velocity", c=colors, cmap='viridis', alpha=0.6)

# Scatter for unspliced expression with black edge color
plt.scatter(x, y_Mu, label="Unspliced expression", edgecolor='black', c=colors, facecolors='none')

# Add labels and legend
plt.title(f"Overlay of Unspliced velocity and Unspliced expression for {gene_name}")
plt.xlabel("isomap_1")
plt.ylabel(f"{gene_name}")
plt.legend()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Retrieve the data
x = adata.obs["isomap_1"]
y_velocity_u = adata.layers["velocity"][:, adata.var_names == gene_name].flatten()
y_Mu = adata.layers["Ms"][:, adata.var_names == gene_name].flatten()

# Get the cluster labels and corresponding colors
clusters = adata.obs["clusters"]
cluster_colors = adata.uns["clusters_colors"]

# Map clusters to colors
cluster_color_map = {cluster: color for cluster, color in zip(clusters.cat.categories, cluster_colors)}
colors = clusters.map(cluster_color_map)

# Plot both curves
plt.figure(figsize=(8, 6))

# Scatter for unspliced velocity with cluster-based colors
plt.scatter(x, y_velocity_u, label="spliced velocity", c=colors, cmap='viridis', alpha=0.6)

# Scatter for unspliced expression with black edge color
plt.scatter(x, y_Mu, label="spliced expression", edgecolor='black', c=colors, facecolors='none')

# Add labels and legend
plt.title(f"Spliced velocity and Spliced expression for {gene_name}")
plt.xlabel("isomap_1")
plt.ylabel(f"{gene_name}")
plt.legend()
plt.show()


In [ ]:
tot = 0
for key in ["nn", "pp", "pn", "np"]:
    tot += adata.layers[key]

tot = tot/4

adata.layers["total_discrepancy"] = tot

sc.pl.umap(adata, color="Tecr", layer="total_discrepancy")

In [ ]:
sc.pl.umap(adata, color="p_cell_uncertainty")

In [ ]:
import scanpy as sc
import torch
from model import VAE

# Load the data
adata_path = "/mnt/data2/home/leonardo/git/multilineage_velocity/benchmark/imVelo/forebrain/imVelo_forebrain.h5ad"
adata = sc.read_h5ad(adata_path)

# Initialize your model
model = VAE(adata)

# Load the checkpoint
model_path = "/mnt/data2/home/leonardo/git/multilineage_velocity/forebrain/model_epoch_20035.pt"
checkpoint = torch.load(model_path)

# Load the model state dictionary from the checkpoint
model.load_state_dict(checkpoint['model_state_dict'])

# Set the model to evaluation mode
model.eval()  # Recommended for inference/evaluation

from dataloaders import setup_dataloaders
_,_,dataloader = setup_dataloaders(adata, 256, train_size=1, split_data=False)

from metrics import compute_and_plot_velocity_coherence
compute_and_plot_velocity_coherence(adata, model, dataloader, 'forebrain', 'Clusters', save_figures=True)